## **UAOPiloto**
Detección de estudiantes que van a desertar en 2 períodos.

TABLAS BASE:  DATOS

TABLAS MYSQL:
1. estudiantes_limpio:  es la tabla ESTUDIANTE pero ahora si limpia
2. periodosRegulares:   es la tabla de periodos y ya viene con un orden que me permite identificar " los siguientes dos periodos"
3.  

# **1.  Conexión a la base de datos**
Nos conectamos a la base de datos MYSQL

In [40]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)

# **2.  crear DATOS**

Se crea DATOS a partir de DISTINCT EstudianteAsignatura(id_estudiante,periodo)

In [41]:
query = """
    SELECT distinct id_estudiante, periodo
    FROM estudianteAsignatura
    ORDER BY id_estudiante, periodo
"""
DATOS = pd.read_sql(query, con=motor)

# NOTA:  falta manejar el caso de cursos de verano...  cuándo marcar al estudainte como desertor?  
# ERIK ya tiene la pregunta 20231009. R/  indica ERIK solo considerar períodos regulares

### **2.1 adicionamos el primer periodo que hizo matricula**
De la tabla EstudiantesMatriculas adicionamos el primer periodo en que el estudiante hizo su primer matricula
EstudiantesMatriculas(id_estudiante, MIN(periodo))

In [42]:
# adicionamos el primer periodo en que ese estudiante hizo una matricula
query = """
    SELECT  id_estudiante, min(periodo) periodo_inicial
    FROM estudianteAsignatura
    WHERE periodo like '%1' or periodo like '%3'
    GROUP BY id_estudiante
"""
temp = pd.read_sql(query, con=motor)
# hacemos join entre DATOS y PRIMER_PERIODO para adicionar el periodo_inicial
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **2.2 Adicionamos fecha_grado, fecha_nacimiento, genero, colegio, ciudad_colegio**
De la tabla Estudiantes_Limpia saco esta información por estudiante y la adicionamos a DATOS

In [43]:
# ahora adicionamos el periodo de grado Y TODAS LAS COLUMNAS REQUERIDAS DESDE ESTUDIANTES_LIMPIO 
# pues ya hice todo el trabajo
# de limpieza.

query = """
    SELECT  id_estudiante, fecha_grado , fecha_nacimiento, genero, colegio, ciudad_colegio
    FROM estudiantes_limpio
"""
temp = pd.read_sql(query, con=motor)

# hacemos join con DATOS para adicionar la fecha de grado
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **2.3 adicionamos el número del periodo (periodo_orden)**

en la tabla PERIODOS está el orden de cada periodo.  vamos a adicionarlos.


In [62]:
query = """
    SELECT  pr.periodo, pr.orden periodo_orden, pr.fecha_inicio periodo_fi, pr.fecha_final periodo_ff
    from periodosRegulares pr
"""
temp = pd.read_sql(query, con=motor)

DATOS = DATOS.merge(temp, on=['periodo'], how='left')


SALVEMOS DATOS para que no nos toque procesar lo anterior

In [65]:
DATOS.to_excel('DATOS.xlsx')

### **2.3 LABEL:  calculamos si dada una fila en d de DATOS, los siguientes 2 periodos no matriculó (ni se graduó)**

Ahora procedemos a generar LABEL: 1- el estudiante desertó.  0- el estudiante no desertó

hacemos un FOR en DATOS
por cada d (en el periodo i)
   miramos si i e i+1 tiene matricula: 
   si si:   lo marcamos como 0 -  
   si no:  miramos si se graduó en i, i+1 o i+2.  
           si si:  marcamos como 0
           si no:  marcamos como 1
   

In [22]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)
DATOS = pd.read_excel('DATOS.xlsx')

In [23]:
# el máximo periodo_orden es 12

from datetime import datetime, timedelta

DATOS['LABEL'] = -1   # asumimos que no se puede calcular

for indice, d in DATOS.iterrows():
    
    id_estudiante = d['id_estudiante']
    periodo_orden = d['periodo_orden']

    # print(id_estudiante, periodo_orden,'fecha_grado:', d['fecha_grado'], 'periodo_fi:', d['periodo_fi'], d['periodo_fi'] + timedelta(days=365))


    if periodo_orden >= 10:  # si es 11 o 12 no podemos aún calcular si si o si no
        d['LABEL'] = -1
        # print('     no se sabe')
    else:
        existe_periodo_1 =  DATOS[(DATOS['id_estudiante'] == id_estudiante) & (DATOS['periodo_orden'] == periodo_orden +1)]
        existe_periodo_2 =  DATOS[(DATOS['id_estudiante'] == id_estudiante) & (DATOS['periodo_orden'] == periodo_orden +2)]

        # si matriculo periodo 1 o periodo 2
        if len(existe_periodo_1)>0 or len(existe_periodo_2)>0:
            
            # como matriculó al menos uno de los dos siguientes periodos, no es desertor.
            DATOS.at[indice, 'LABEL'] = 0  

        else: 

            # si entra aquí es porque no matriculo ninguno de los dos siguientes períodos

            # si no tiene fecha de grado, es desertor.
            if pd.isna(d['fecha_grado']):
                DATOS.at[indice, 'LABEL'] = 1
            else:  
                
                # si sí tiene fecha de grado, pero es posterior a 1 año, es desertor.
                if d['fecha_grado'] > d['periodo_fi'] + timedelta(days=365):  
                    DATOS.at[indice, 'LABEL'] = 1
                else:
                    # si la fecha de grado ocurre antes de un año, NO es desertor.
                    DATOS.at[indice, 'LABEL'] = 0


In [24]:
DATOS.columns

Index(['Unnamed: 0', 'id_estudiante', 'periodo', 'periodo_inicial',
       'fecha_grado', 'fecha_nacimiento', 'genero', 'colegio',
       'ciudad_colegio', 'periodo_orden', 'periodo_fi', 'periodo_ff', 'LABEL'],
      dtype='object')

# **3.  Adicionar variables de interés**

El excel HIPOTESIS vs VARIABLES va explicando qué variables se van a ir adicionando

### **3.1 Resultados por áreas icfes**
Se adiciona el resultado en cada área que obtuvo el estudiante, incluyendo el resultado en ICFES_ANTIGUO
Si hay nulo se coloca -100

In [16]:
query = """
    SELECT  id_estudiante, IFNULL(icfes_antiguo,-100) icfes_antiguo, IFNULL(biologia,-100) biologia, IFNULL(matematica,-100) matematica, 
			IFNULL(filosofia,-100) filosofia, 
            IFNULL(fisica,-100) fisica, IFNULL(historia,-100) historia, IFNULL(quimica,-100) quimica, IFNULL(lenguaje,-100) lenguaje, 
            IFNULL(geografia,-100) geografia, IFNULL(idioma,-100) idioma, IFNULL(interdisciplinar,-100) interdisciplinar
    FROM icfes;
"""
temp = pd.read_sql(query, con=motor)
# hacemos join (temporal) contra datos y vamos a mirar cómo podemos agrupar cada variable frente a LABEL
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **3.2 Distribución notas obtenidas semestres anteriores**

Para poder obtener esta información correctamente es conveniente NORMALIZAR la tabla EstudianteAsignatura
en 3 tablas:

EstudianteAsignatura:  info del estudiante y la asignatura que matrículo. PK= (periodo, id_estudiante, cod_asig) 
EA_grupo:  que tiene como pk (periodo, id_estudiante, cod_asig, grupo)
EA_docentes:  que tiene como pk (periodo, id_estudiante, cod_asig, grupo, cedula)

Y también se creó la columna nivel_nota en estudianteasignatura que nos dice cuantas de ese tipo sacó. 
if definitiva null ->  nivel_nota = -1
if definitiva menor a 3.0 -> nivel_nota = 1
if definitiva entre 3 y menor a 3.8 -> nivel_nota = 2
if definitiva mayor o igual a 3.8 -> nivel_nota = 3

por cada fila d en DATOS:
   obtener la cantidad de notas nulas, bajas, medias, altas que obtuvo en períodos anteriores

In [36]:
query = """
    SELECT  ae.id_estudiante, ae.periodo, pr.orden periodo_orden, ae.nivel_nota, count(*) cuantas
    FROM estudianteasignatura ae, periodosregulares pr
    WHERE pr.periodo = ae.periodo
    group by ae.id_estudiante, ae.periodo, pr.orden, ae.nivel_nota
"""
temp = pd.read_sql(query, con=motor)

# temp = temp.pivot(index=["id_estudiante", "orden"], columns="nivel_nota", values="cuantas").reset_index()
# temp = temp.fillna(0)
print(temp.columns)


Index(['id_estudiante', 'periodo', 'periodo_orden', 'nivel_nota', 'cuantas'], dtype='object')


In [31]:
DATOS.columns

Index(['Unnamed: 0', 'id_estudiante', 'periodo', 'periodo_inicial',
       'fecha_grado', 'fecha_nacimiento', 'genero', 'colegio',
       'ciudad_colegio', 'periodo_orden', 'periodo_fi', 'periodo_ff', 'LABEL'],
      dtype='object')

In [76]:
# seleccionarmos solo id_estudiante, periodo y periodo_orden de DATOS
temp2 = DATOS[['id_estudiante','periodo','periodo_orden']]

# hacemos left join con temp
temp2 = temp2.merge(temp, on=['id_estudiante'], how='left')

# dejamos solo las filas donde periodo_orden_x > periodo_orden_y pues queremos mirar al periodo cuantas
# materias tuvo sin nota, cuantas tuvo con nivel 1, cuantas con nivel 2 y cuantas con nivel 3 de periodos
# anteriores
temp2 = temp2[temp2['periodo_orden_x'] > temp2['periodo_orden_y']]

# ahora agrupamos para saber el total de materias con nulo, 1,2 o 4 de periodos anteriores.
temp2 = temp2.groupby(['id_estudiante','periodo_x','nivel_nota'])['cuantas'].sum().reset_index()

# pivoteamos
temp2 = temp2.pivot(index=['id_estudiante','periodo_x'], columns='nivel_nota', values='cuantas').reset_index()

# renombramos y llenamos con ceros
temp2 = temp2.rename(columns={'periodo_x':'periodo', '-1':'nivel_menos_1','1':'nivel_1','2':'nivel_2','3':'nivel_3'})
temp2 = temp2.fillna(0)
temp2

nivel_nota,id_estudiante,periodo,nivel_menos_1,nivel_1,nivel_2,nivel_3
0,2157111,201803,0.0,0.0,1.0,5.0
1,2157111,201901,0.0,0.0,1.0,11.0
2,2157129,201803,0.0,0.0,0.0,6.0
3,2157129,201901,0.0,0.0,1.0,11.0
4,2157130,201803,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...
585,2231780,202303,0.0,0.0,1.0,7.0
586,2231781,202303,0.0,0.0,0.0,8.0
587,2231877,202303,0.0,0.0,5.0,3.0
588,2232058,202303,0.0,0.0,1.0,7.0


In [85]:
# ahora hacemos el join con DATOS
DATOS = DATOS.merge(temp2, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.fillna(0)
DATOS

,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,periodo_orden,...,periodo_ff,LABEL,nivel_menos_1_x,nivel_1_x,nivel_2_x,nivel_3_x,nivel_menos_1_y,nivel_1_y,nivel_2_y,nivel_3_y
0,0,2136689,201803,201803.0,0,1989-12-14,MAS,LICEO REMINGTON,MEDELLIN,2,...,2018-12-31,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2146669,201801,201801.0,2018-12-01 00:00:00,1996-08-28,FEM,COLEGIO REYES CATOLICOS,CALI,1,...,2018-06-30,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2146707,201801,201801.0,0,1997-03-09,FEM,COLEGIO SAN FRANCISCO DE ASIS - CALI,CALI,1,...,2018-06-30,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2146709,201801,201801.0,2019-04-06 00:00:00,1997-04-20,MAS,LICEO ANGLO DEL VALLE,CALI,1,...,2018-06-30,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2146710,201801,201801.0,0,1996-08-11,FEM,COLEGIO MIXTO SAN VICENTE,CALI,1,...,2018-06-30,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,827,2236079,202303,202303.0,0,2001-03-18,MAS,CENTRO DE EDUCACION EN TECNOLOGIA CENTEC,CALI,12,...,2023-12-31,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
828,828,2236229,202303,202303.0,0,2001-02-25,FEM,COLEGIO MONTERREY,BOGOTÁ D.C.,12,...,2023-12-31,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
829,829,2236496,202303,202303.0,0,2001-12-19,MAS,COLEGIO BOSTON DE TEUSAQUILLO,BOGOTÁ D.C.,12,...,2023-12-31,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
830,830,2236497,202303,0.0,0,2006-05-22,FEM,LICEO LA ENSEÑANZA,PALMIRA,12,...,2023-12-31,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
